In [ ]:
from ibicus.debias import QuantileMapping
import xarray as xr
from pathlib import Path

from deepdown.utils.data_loader import DataLoader
from deepdown.utils.debiaser_utils import get_ibicus_var_name, prepare_for_ibicus
from deepdown.config import Config
from deepdown.bias_correction_ibicus import correct_bias
import matplotlib.pyplot as plt

from argparse import Namespace
cli_args = Namespace(config_file="../config.yaml")
config = Config(cli_args)
conf = config.config


In [ ]:
dir_path = Path(conf.path_output)
input_data_clim_debiased = xr.open_dataset(dir_path / "input_data_clim_debiased.nc")
input_data_clim_debiased

In [ ]:
plt.figure(figsize=(10, 6))
input_data_clim_debiased['t'][:,20,20].plot(label='t')
input_data_clim_debiased['t_deb'][:,20,20].plot(label='t debiased')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.legend()
plt.show()

In [ ]:

# Load target data for the historical period
target_data_hist = DataLoader(path_tmp=conf.path_tmp)
target_data_hist.load(conf.period_hist_start, conf.period_hist_end,
                        conf.path_targets)

# Load input data (e.g. climate model) for the historical period
input_data_hist = DataLoader(path_tmp=conf.path_tmp)
input_data_hist.load(conf.period_hist_start, conf.period_hist_end, conf.path_inputs)

# Load input data (e.g. climate model) for the future period
input_data_clim = DataLoader(path_tmp=conf.path_tmp)
input_data_clim.load(conf.period_clim_start, conf.period_clim_end, conf.path_inputs)

# Coarsen the target data to the resolution of the input data
target_data_hist.coarsen(
    x_axis=input_data_hist.data.x, y_axis=input_data_hist.data.y,
    from_proj='CH1903_LV95', to_proj='WGS84')

# Bias correct each input variable
for var_target, var_input in zip(conf.target_vars, conf.input_vars):
    # Map to the ibicus variable names
    var_ibicus = get_ibicus_var_name(var_target)

    # Prepare data for ibicus
    target_array_hist = prepare_for_ibicus(target_data_hist, var_target)
    input_array_hist = prepare_for_ibicus(input_data_hist, var_input)
    input_array_clim = prepare_for_ibicus(input_data_clim, var_input)

    debiaser = QuantileMapping.from_variable(var_ibicus)
    debiased_ts = debiaser.apply(
        target_array_hist,
        input_array_hist,
        input_array_clim)

    debiased_var = f"{var_input}_deb"
    input_data_clim.data = input_data_clim.data.assign(
        **{debiased_var: (('time', 'y', 'x'), debiased_ts)})

# Save the debiased dataset to a NetCDF file
output_path = Path(conf.path_output)
file_out = output_path / "input_data_clim_debiased.nc"
input_data_clim.data.to_netcdf(file_out)